# Einführung in die Materialmodellierung - Übung 02 Rheologie
## Installation
 * bitte installieren sie Plots, Plotly über den Package Manager von Julia
 * gehen sie dafür in das REPL (Julia Shell), drücken sie die Taste "]" und tippen sie "add Plots, Plotly"
 * danach importieren sie Plots und ändern das Backend auf plotly

In [11]:
using Plots;
plotly()

Plots.PlotlyBackend()

In [12]:
t = collect(0:1:100);

**Nun definieren wir Funktionen, die den vorgegebenen $\varepsilon$ Verlauf erzeugen**

In [13]:
function constant(max::Float64, t::Array)
    load = max * ones(length(t))
    return load
end

function constant_cycle(max::Float64, t::Array)
    T = length(t)
    load = max * ones(T)
    load[(T÷4+1):(T÷2)] .= 0
    load[(T*3÷4+1):T] .= 0
    return load
end

function sinus(max::Float64, t::Array)
    load = max * sin.(8*pi/100*t)
    return load
end;

**Visualisierung der Belastung**

Der Plot ist interaktiv, Sie können auf der rechten Seite auswählen, welche Funktion angezeigt werden soll.

Möchten Sie eine Funktion entfernen, klicken sie auf den Namen in der Legende

In [14]:
plot(constant(0.1, t), ylabel="ε oder σ", xlabel="Zeit t", label="konstant", line=5)
plot!(constant_cycle(0.1, t), line=5, label="zyklisch konstant")
plot!(sinus(0.1, t), line=5, label="sinus")

<!DOCTYPE html>
 
 
 Plots.jl

**Zener$_M$ Modell / Standard Modell in Maxwell Repräsentation**
$$\sigma_{n+1}=\frac{\eta_M\sigma_n + (E_H+E_M)\eta_M(\varepsilon_{n+1} - \varepsilon_n)+E_H E_M\Delta t \varepsilon_{n+1}}{\eta_M + E_M \Delta t}$$
$$\varepsilon_{n+1} = \frac{(E_H + E_M)\eta_M\varepsilon_n+\eta_M(\sigma_{n+1} - \sigma_n) + E_M \Delta t \sigma_{n+1}}{(E_H+E_M)\eta_M+E_H E_M \Delta t}$$

In [15]:
function zenerₘ_σ(Eₘ::Float64, Eₕ::Float64, ηₘ::Float64, ε::Array, t::Array)
    σ = zeros(length(t))
    Δε = zeros(length(t))
    σ[1] = ε[1]*(Eₕ+Eₘ)
    for i in 1:length(t)-1
        Δt = t[i+1] - t[i]
        Δε[i+1] = ε[i+1] - ε[i]
        σ[i+1] = (ηₘ*σ[i] + (Eₕ+Eₘ)*ηₘ*Δε[i+1] + Eₕ*Eₘ*Δt*ε[i+1])/(ηₘ+Eₘ*Δt)
    end
    return σ, Δε;
end

function zenerₘ_ε(Eₘ::Float64, Eₕ::Float64, ηₘ::Float64, σ::Array, t::Array)
    ε = zeros(length(t))
    Δσ = zeros(length(t))
    ε[1] = σ[1]/(Eₕ+Eₘ)
    for i in 1:length(t)-1
        Δt = t[i+1] - t[i]
        Δσ[i+1] = σ[i+1] - σ[i]
        ε[i+1] = ((Eₕ+Eₘ)*ηₘ*ε[i] + ηₘ*Δσ[i+1] + Eₘ*Δt*σ[i+1])/(ηₘ*(Eₕ+Eₘ) + Eₕ*Eₘ*Δt)
    end
    return ε, Δσ
end

zenerₘ_ε (generic function with 1 method)

**Relaxationsversuch**

In [26]:
Eₕ = 20.
Eₘ = 100.
ηₘ = 1000.
σ_ZenerM, Δε_ZenerM = zenerₘ_σ(Eₘ, Eₕ, ηₘ, constant(0.01, t), t);
plot(t,σ_ZenerM, ylabel="σ", xlabel="Zeit t", label="Zener M", line=5)

<!DOCTYPE html>
 
 
 Plots.jl

**Kriechversuch**

In [31]:
Eₕ = 20.
Eₘ = 100.
ηₘ = 100.
ε_ZenerM, Δε_ZenerM = zenerₘ_ε(Eₘ, Eₕ, ηₘ, constant(1., t), t);
plot(t,ε_ZenerM, ylabel="ε", xlabel="Zeit t", label="Zener M", line=5)

<!DOCTYPE html>
 
 
 Plots.jl

**Testen, ob die Implementierung die Initial und Endbedingungen dieses Modells erfüllt, um Konsistenz zu überprüfen**

In [94]:
σ₀ = Eₕ * ε_max 

2.0

In [96]:
σ_∞ = (Eₕ*Eₘ)/(Eₕ + Eₘ) * ε_max

1.666666666666667

In [108]:
using Test
@test isapprox(σ₀, σ_ZehnerM[1], atol=1e-5)
@test isapprox(σ_∞, σ_ZehnerM[end], atol=1e-5)

Test Passed

**Implementieren Sie das Modell, welches in der Übung hergeleitet wurde**

In [ ]:
function modell()
    
end